# 4. Further Advanced NumPy Modelling Examples

This notebook will involve no direct teaching - instead we are going to attempt to solve a number of difficult problems. These problems will attempt to range over a number of inter-disciplinary fields. Don't worry if you are not able to complete them all within the time of the workshop - they are meant to stretch your abilities, gain some useful NumPy experience and grow some inter-disciplinary knowledge. For these tasks we require that you **only** use NumPy arrays as this is considerably faster and the only tractable method in later examples.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d.axes3d import Axes3D
plt.rcParams['figure.figsize'] = 15, 10
%matplotlib inline

# Tasks

## Stochastic Differential Equations

A *random walk* or *Brownian process* is a way of modelling error introduced by uncertainty into a differential equation. This has a range of applications, primarily in time-series modelling of uncertainty such as stock price in *Computational Finance*. The random variable representing the walk is denoted $W$. A single realization of the walk is written $W(t)$. We will assume:
1. Perfect knowledge at the beginning of the walk: $W(0)=0$
2. Every step is *independent* of every other step.

This leads us to a *discrete random walk*; given timesteps $t=0,\dots,T$ separated by a uniform step size $\delta t$ we have a single realization of the *random walk*:

$$
\Delta W_i=\sqrt{\delta t}\cal N[0, 1]
$$

where $\cal N[0, 1]$ refers to number generated from a normal distribution with mean = 0, and variance = 1. The walk $W$ is then calculated as the cumulative summation of all of the changes in $W$; $\Delta W$, the partial sum up to $i$ is hence:

$$
W_i = \left(\sum_{j=0}^{i-1} \Delta W_j \right)
$$

We can apply this walk to **stochastic differential equations** which are written in the form:

$$
\frac{\text{d}X}{\text{d}t}=f(X)+g(X)\frac{\Delta W}{\text{d}t}
$$

which has a formal solution:

$$
X(t)=X_0 + \int_0^1 f(X(s))\text{d}s + \int_0^1 g(X(s))\Delta W_s
$$

This can be approximated *numerically* using the **Euler-Maruyama** method, written in discrete subscript form:

$$
X_{n+1}=X_n+f_n\delta t+g_n \Delta W_n
$$


### Task 1.

Simulate a single realisation of a *Brownian process* using a step of $\delta t =\frac{1}{N}$ for $N=500$. $N$ is the number of points we will sample in linspace(). Go up to $T \in [0, 1]$. Use a fixed seed using np.random.seed(int). Plot $t$ against $W$.

In [ ]:
# your codes here

### Task 2.

Simulate $10$ and $10^5$ realisations $M$ of a *Brownian process* using the same step size, and $T=10^4$. Plot $t$ against mean $W$ with respect to each timestep $t$.

In [ ]:
# your codes here

### Task 3.

We have the **stochastic differential equation**:

$$
\text{d}X(t)=\lambda X(t)\delta t + \mu X(t)\Delta W(t), \qquad X(0)=1
$$

Write the function `euler_maruyama_step()` which takes $X_n$, $\delta t$, $\Delta W_n$, $\lambda=2$ and $\mu=1$ and returns $X_{n+1}$, where $f_n=\lambda X_n$ and $g_n=\mu X_n$.

In [ ]:
# your codes here

### Task 4.

Write the function `euler_maruyama()` which takes $N$ (the number of points), $\delta t$, a random walk $\Delta W$, and an initial position $X(0)$. This method should return an array of $X$ in time. This function should call `euler_maruyama_step()` every iteration. Run `euler_maruyama()` with $N=100$, $\lambda=2$, $\mu=1$, $X(0)=1$ from time $t=0, \dots, T$. 

In [ ]:
# your codes here

### Task 5.

The exact solution to the equation in *Task 3* is:

$$
X(t)=X(0)\exp \left[\left(\lambda - \frac{1}{2}\mu^2\right)t + \mu W(t)\right]
$$

Calculate the exact solution using the same timesteps for `euler_maruyama()`. Plot the exact solution (labelled) and the numerical solution (labelled) as a line plot to see the divergence. 

In [ ]:
# your codes here

## Molecular Dynamics

In molecular dynamics we are concerned with applying classical Newtonian laws to particles in cartesian space. Each particle has a location $\vec{X}_i$ which obeys:

$$
m_i \frac{d^2}{dt^2}\vec{x}_i = m_i\vec{A}_i = -\nabla V \left(\vec{x}_1, \dots, \vec{x}_N \right)
$$

For this example we will set $m_i=1$ for all $i$ to simplify things. We can think of every particle having a position within 3-D space as a 3-value vector:

$$
\vec{x}_i = (x_1, x_2, x_3)
$$

The change in position with respect to time can be thought of as the *speed* of the molecule, and likewise the change in speed with respect to time is considered the *acceleration* of the molecule:

$$
\frac{d}{dt}\vec{x}_i=\vec{x}_i'=\vec{v}_i, \qquad \frac{d^2}{dt^2}\vec{x}_i=\vec{x}_i''=\vec{A}_i
$$

This gives us a computational domain, once translated to the centre-of-mass, of $\vec{x} \in [-0.5,0.5]^3$. In addition to this, we want to model the *interaction* between particles - the simplest possibility is that the interaction potential $V$ is the sum of pairwise interactions:

$$
V \left(\vec{x}_1, \dots, \vec{x}_N \right)=\sum_i \sum_{j<i} \phi \left(L \left| \vec{x}_i - \vec{x}_j \right| \right)
$$

where $L$ refers to the reference length of the coordinate space. The most commonly used pairwise interaction potential is the *Lennard-Jones* potential:

$$
\phi(r)=4\epsilon \left[\left(\frac{\sigma}{r}\right)^{12} - \left(\frac{\sigma}{r}\right)^6 \right]
$$

where $\epsilon$ is the depth of the potential well, $\sigma$ is the finite distance at which the inter-particle potential is 0, and $r$ is the distance between the particles. At the distance where the potential reaches it's minimum, $r_m = 2^{1/6}\sigma$. The $r^{-12}$ term describes repulsuion between molecules at short ranges due to overlapping electron orbitals, and the $r^{-6}$ term describes long-range attraction due to *van der Waals* forces. However to prevent the potential for extending to infinite distance, we use a cutoff: $\phi(r) \to \phi(r)-\phi(R_c)$, where $R_c=2.5$ is a constant, when $\sigma=1$, and $\epsilon=m_i$. Thus the derivative of our pairwise interaction potential function becomes:

$$
\frac{d}{dr}\phi(r) = 24\left[2\left(\frac{1}{r}\right)^{14}-\left(\frac{1}{r}\right)^8\right]
$$

thus we can calculate the *acceleration* of all the particles at each timestep as:

$$
\vec{A}_i = \sum_{j>i}(\vec{x}_i - \vec{x}_j)\frac{d}{dr}\phi(r)-\sum_{j<i}(\vec{x}_j - \vec{x}_i)\frac{d}{dr}\phi(r)
$$

Note that Newton's third law means that the force $F$ on *particle* $i$ from *particle* $j$ must be exactly the opposite to the force on *particle* $j$ from *particle* $i$. This means that:

$$
\vec{A}_i = \vec{A}_i + (\vec{x}_i - \vec{x}_j) \frac{d}{dr}\phi(r) \\
\vec{A}_j = \vec{A}_j - (\vec{x}_i - \vec{x}_j) \frac{d}{dr}\phi(r)
$$

This then is only part of the whole picture, once we have calculated *acceleration*, we need to evolve the movement of particles across time. One of the powerful algorithms for this is the **Velocity Verlet algorithm**:

1. Calculate next step $\vec{x}^{n+1}=\vec{x}^n + \delta t \ \vec{v}^n + \frac{1}{2}\delta t^2 \vec{A}^n$;
2. Apply boundary checks to $\vec{x}^{n+1}$;
3. Calculate vstar $\vec{v}^{*}=\vec{v}^n + \frac{1}{2}\delta t \ \vec{A}^n$;
4. Derive $\vec{A}^{n+1}$ from the interaction potential using $\vec{x}^{n+1}$;
5. Calculate $\vec{v}^{n+1}=\vec{v}^{*}+\frac{1}{2} \delta t \ \vec{A}^{n+1}$

This will evolve all of the particles in 3-D space over $\infty$ steps.

### Task 1.

Write a function `lennard_jones_potential()` that given $r$ as input, computes the derivative of $\phi(r)$.

In [ ]:
# your codes here

### Task 2.

The algorithm for acceleration is as follows:

1. Initialise $\vec{A}_i=0$, $N=len(\vec{x})$, $P=dim(\vec{x})$, $i=0$, $j=0$.
1. For $i, \dots, N$
    1. Calculate $\vec{x}'=\vec{x}_i-\vec{x}$
    2. Compute $\vec{x}' = \vec{x}' - sign(\vec{x}')L \qquad$ where $\left|\vec{x}'\right| > \frac{L}{2}$, scaling by reference length.
    3. For $j = i+1, \dots, N$
        1. Calculate the magnitude of the coordinate changes, $r_{ij}=\sqrt{\vec{x}_j' \cdot \vec{x}_j'}.$
        2. If $r_{ij}<R_c$
            1. Calculate $ \frac{d}{dr}\phi(r_{ij})$
            2. Calculate $\vec{A}_i$.
            3. Calculate $\vec{A}_j$.
1. return $\vec{A}$.

Implement the function `acceleration()`, passing in $\vec{x}$, $L=10$, and $R_c=2.5$. Test `acceleration()` with two particles in an (3,2) numpy array, as $[4,0,0]$ and $[4+2^{\frac{1}{D}},0,0]$, which should lead to no/little acceleration with $D=6$. Compare this to $D=12$ to see the change in acceleration.

In [ ]:
# your codes here

### Task 3.

Write the algorithm `verlet()` which takes as input $\vec{x}^n$, $\vec{v}^n$, $\vec{A}^n$, $\delta t$ and $L$, and returns $\vec{x}^{n+1}$, $\vec{v}^{n+1}$, $\vec{A}^{n+1}$. To boundary check, ensure that:

$$
\vec{x}^{n+1} = \vec{x}^{n+1} + L, \qquad \vec{x}^{n+1} < 0 \\
\vec{x}^{n+1} = \vec{x}^{n+1} - L, \qquad \vec{x}^{n+1} > L
$$ .

In [ ]:
# your codes here

### Task 4.

Using the positional input:

$$
\vec{x}=\left[\begin{matrix}
    4 & 0 & 0 \\
    4+2^{\frac{1}{12}}&  0 &0
\end{matrix}\right]
$$

Generate 500 steps of the *velocity verlet* and at each timestep plot the scatterplot, using $\delta t=0.01$, $R_c=2.5$ and $L=10$.

In [ ]:
# your codes here

### Task 5.

The total kinetic energy of the particle can be calculated as:

$$
E_{kinetic,i}=\frac{1}{2}L^2 \left| \left| \vec{v}_i \right| \right|_2^2
$$

and henceforth the mean temperature of the system can be estimated from the total kinetic energy of each particle:

$$
T = \frac{2}{3N}\sum_{i=1}^N E_{kinetic,i}
$$

Write a function `calc_temperature()` which takes as inputs $\vec{v}$ and $L$, and returns the mean temperature of the system. Integrate this with the previous task and plot the mean temperature as a function of time (timesteps).

In [ ]:
# your codes here